<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 2.1 - [PROJECT] - Customized Notification Reports Using Snowflake Snowpark for Python

In this notebook, you will learn how to create custom reports using Snowpark API and Snowflake Notifications

## Learning Objectives

By the end of this classroom, you should be able to:
- Creating Snowflake Snowpark code to generate custom Reports
- Understanding Snowflake Notifications and Generating Mail Notification
- Creating Snowflake Snowpark Stored Procedure to Create a Sproc

In [ ]:
# Lets Get Started with Snowpark For Python
from assets.config import connection_builder
session = connection_builder()

## Section 1 - Creating Snowpark Code to Generate Customized Report

In this module, we will learn how we can generate custom query reports and send them as mail alerts using Snowflake Snowpark For Python.

- We will first create a sample code to generate the dataset we want as a mail report.
- Once its generated we will use Email Notification to trigger the report. 
- Lastly we will encapsulate the code to register as stored procedure on snowflake

---

### [Sending Email Notifications](https://docs.snowflake.com/en/user-guide/email-stored-procedures)

This feature uses the notification integration object, which is a Snowflake object that provides an interface between Snowflake and third-party services (e.g. cloud message queues, email, etc.). A single account can define a maximum of ten email integrations and enable one or more simultaneously.


In [ ]:
from snowflake.snowpark.functions import lit, concat_ws
# Generating Sample Data Using SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER 
src_data = session.table('SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER').limit(1000)
src_data.write.save_as_table(table_name='CUSTOMERS',mode='overwrite',table_type='transient')

# Transforming the sample data for sending Notification
main_data = session.read.table('CUSTOMERS')
main_data.update({'C_EMAIL_ADDRESS': None,'C_PREFERRED_CUST_FLAG':'N'}, (main_data.c_birth_country == 'NEPAL'))

notification_dataset = main_data.filter(  (main_data.c_email_address.isNull())
                                        & (main_data.c_birth_country.isin('NEPAL','INDIA','ALGERIA'))
                                        ).select(main_data.C_CUSTOMER_ID,
                                                 main_data.C_FIRST_NAME,
                                                 main_data.C_LAST_NAME,
                                                 main_data.C_EMAIL_ADDRESS,
                                                 main_data.C_PREFERRED_CUST_FLAG,
                                                 concat_ws(lit('-'),main_data.C_BIRTH_YEAR,main_data.C_BIRTH_MONTH,main_data.C_BIRTH_DAY).alias('DOB')
                                                 ).with_column('STATUS',lit('FLAGED'))
html_text = notification_dataset.to_pandas().to_html()

# *********** *********** *********** *********** *********** *********** *********** ***********  
# ***********    User need to provide the Mail Integration which they want to use     *********** 
# *********** *********** *********** *********** *********** *********** *********** ***********  
integration = 'MAIL_NOTIFICATION'
mail_subject = 'Snowflake Report For Flagged Dataset'
mails = session.sql('desc integration '+integration)
for emails in mails.toLocalIterator():
    if emails[0]=='ALLOWED_RECIPIENTS':
        to_list = emails[2]
session.call('SYSTEM$SEND_EMAIL',
             integration,
             to_list,
             mail_subject,
             html_text,
             'text/html')


In [ ]:
# Creating a Stored Proc for the Above Report
import snowflake.snowpark
from snowflake.snowpark.types import StringType

def flagged_dataset_report(session:snowflake.snowpark.Session, mail_integration:str)->str:
    from snowflake.snowpark.functions import lit, concat_ws
    # Generating Sample Data Using SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER 
    src_data = session.table('SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER').limit(1000)
    src_data.write.save_as_table(table_name='CUSTOMERS',mode='overwrite',table_type='transient')

    # Transforming the sample data for sending Notification
    main_data = session.read.table('CUSTOMERS')
    main_data.update({'C_EMAIL_ADDRESS': None,'C_PREFERRED_CUST_FLAG':'N'}, (main_data.c_birth_country == 'NEPAL'))

    notification_dataset = main_data.filter(  (main_data.c_email_address.isNull())
                                            & (main_data.c_birth_country.isin('NEPAL','INDIA','ALGERIA'))
                                            ).select(main_data.C_CUSTOMER_ID,
                                                    main_data.C_FIRST_NAME,
                                                    main_data.C_LAST_NAME,
                                                    main_data.C_EMAIL_ADDRESS,
                                                    main_data.C_PREFERRED_CUST_FLAG,
                                                    concat_ws(lit('-'),main_data.C_BIRTH_YEAR,main_data.C_BIRTH_MONTH,main_data.C_BIRTH_DAY).alias('DOB')
                                                    ).with_column('STATUS',lit('FLAGED'))
    html_text = notification_dataset.to_pandas().to_html()

    mail_subject = 'Snowflake Report For Flagged Dataset'
    mails = session.sql('desc integration '+mail_integration)
    for emails in mails.toLocalIterator():
        if emails[0]=='ALLOWED_RECIPIENTS':
            to_list = emails[2]
    session.call('SYSTEM$SEND_EMAIL',
                mail_integration,
                to_list,
                mail_subject,
                html_text,
                'text/html')
    return 'success'

session.sproc.register(func=flagged_dataset_report,
                       name='sp_email_report',
                       return_type=StringType(),
                       input_types=[StringType()],
                       is_permanent=True,
                       replace=True,
                       stage_location='@TEST_STAGE',
                       execute_as='CALLER',
                       packages=['snowflake-snowpark-python'])

# *********** *********** *********** *********** *********** *********** *********** ***********  
# ***********    User need to provide the Mail Integration which they want to use     *********** 
# *********** *********** *********** *********** *********** *********** *********** ***********  
session.call('sp_email_report','MAIL_NOTIFICATION')